In [1]:
!cd /content

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"prakratimathur","key":"42b60eebd58437b01f4470d419063478"}'}

In [11]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
!mv ./kaggle.json ~/.kaggle

mv: cannot stat './kaggle.json': No such file or directory


In [13]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [7]:
!pip install kaggle

In [8]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 84% 68.0M/80.9M [00:00<00:00, 216MB/s]
100% 80.9M/80.9M [00:00<00:00, 231MB/s]


In [9]:
!unzip /content/sentiment140.zip



Archive:  /content/sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [10]:
!pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.8 MB/s eta 0:00:00


In [14]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
import contractions
import re
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [15]:
import pandas as pd
from keras.layers import TextVectorization
import numpy as np
from keras.layers import Input,Embedding,LSTM,GRU,Dense
from keras.models import Sequential
import pickle
import locale
import tensorflow as tf
import os

In [16]:
locale.getpreferredencoding=lambda:"UTF-8"


In [17]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
nltk.download("wordnet")

#WordNet is a lexical database for the English language that provides information about words, including their meanings, semantic relationships, and more

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
cpu_count()


2

In [21]:
data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",encoding="latin-1",header=None)

In [22]:
data.shape

(1600000, 6)

In [23]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [24]:
data.rename(columns={0:"target",1:"ids",2:"date",3:"flag",4:"user",5:"text"},inplace=True)

In [25]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [26]:
data["text"].iloc[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [27]:
reduced_data=data.drop(labels=data.columns[1:5],axis=1)

In [28]:
reduced_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [29]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [30]:
def normalize_tweet(tweet_text):
  return tweet_text.lower()

In [31]:
with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_data["text"]=list(pool.map(normalize_tweet,list(reduced_data["text"])))

In [32]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he can't update his facebook by ...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [33]:
def expand_contractions(tweet_text):
  return contractions.fix(tweet_text)

In [34]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(expand_contractions,list(reduced_data["text"])))



In [35]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he cannot update his facebook by...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it is not behaving at all...
Name: text, dtype: object

In [36]:
regex_pattern = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'


In [37]:
def remove_noisy_tokens(tweet_text):

  return re.sub(pattern=regex_pattern,repl=" ",string=tweet_text)

In [38]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_noisy_tokens,list(reduced_data["text"])))

In [39]:
reduced_data["text"].head()

0      twitpic com  y zl awww that is a bummer you ...
1    is upset that he cannot update his facebook by...
2          managed to save   the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4      it is not behaving at all i am mad why am i ...
Name: text, dtype: object

In [40]:
def remove_remaining_noisy_tokens(tweet_text):

  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',repl=" ",string=tweet_text)

In [41]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_data["text"])))

In [42]:
reduced_data["text"].head()

0     twitpic com   zl awww that is   bummer you sh...
1    is upset that he cannot update his facebook by...
2            managed to save the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4     it is not behaving at all   am mad why am   h...
Name: text, dtype: object

In [43]:
def tokenize_tweet_text(tweet_text):

  return word_tokenize(tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(tokenize_tweet_text,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

In [ ]:

def is_stopword(token):
    return not(token in en_stop_words)

In [ ]:
def remove_stopwords(tweet_text):

  return list(filter(is_stopword,tweet_text))

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_stopwords,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wn.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wn.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wn.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wn.ADV)
    else:
        return (treebank_tag[0],wn.NOUN)

In [ ]:
def get_pos_tag(tweet_text):

    return list(map(get_wnet_pos_tag,pos_tag(tweet_text)))

In [ ]:
print(get_pos_tag(tweet_text))

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
reduced_data["text"] = list(map(get_pos_tag,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_token(token_pos_tuple):

    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [ ]:

def lemmatize_text(tweet_text):

    if len(tweet_text) > 0:
        return list(map(lemmatize_token,tweet_text))
    else:
        return [""]

In [ ]:
reduced_data["text"] = list(map(lemmatize_text,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()


In [ ]:
with open("/content/tokenized_data.pkl","wb") as file_handle:

  pickle.dump(reduced_data,file_handle)

In [ ]:
!mv /content/tokenized_data.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/tokenized_data.pkl /content

In [ ]:
with open("/content/tokenized_data.pkl","rb") as file_handle:

  reduced_data = pickle.load(file_handle)

In [ ]:
reduced_data.head()

In [ ]:
max_tokens = 30000
max_sequence_len = max(list(reduced_data["text"].apply(lambda x: len(x))))

In [ ]:
reduced_rows_idx = np.argwhere(np.array(reduced_data["text"].apply(lambda x: len(x))) >= 10)

In [ ]:
reduced_rows_idx = np.argwhere(np.array(reduced_data["text"].apply(lambda x: len(x))) >= 10)

In [ ]:
reduced_data["text"] = list(map(lambda x: " ".join(x), list(reduced_data["text"])))

In [ ]:
reduced_data.head()

In [ ]:
filtered_reduced_data = reduced_data[reduced_data["text"] != ""]

In [ ]:
filtered_reduced_data.shape

In [ ]:
vectorize_layer = TextVectorization(max_tokens=max_tokens,output_sequence_length=max_sequence_len)

In [ ]:
vectorize_layer.adapt(data=np.array(filtered_reduced_data["text"]))

In [ ]:
vectorize_layer.adapt(data=np.array(filtered_reduced_data["text"]))

In [ ]:
with open("/content/token_integer_mapping.pkl","wb") as file_handle:

  pickle.dump({"config":vectorize_layer.get_config(),
               "weights":vectorize_layer.get_weights()},file_handle)

In [ ]:
!mv /content/token_integer_mapping.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/token_integer_mapping.pkl /content

In [ ]:
with open("/content/token_integer_mapping.pkl","rb") as file_handle:

  vectorize_layer_attributes = pickle.load(file_handle)

In [ ]:
loaded_vectorization_layer = TextVectorization.from_config(vectorize_layer_attributes["config"])

In [ ]:
loaded_vectorization_layer.set_weights(vectorize_layer_attributes["weights"])

In [ ]:

def vectorize_tweet(raw_tweet):

  return loaded_vectorization_layer(raw_tweet).numpy()

In [ ]:

vectorized_tweets = list(filtered_reduced_data["text"].apply(vectorize_tweet))

In [ ]:
vectorized_train_tweets = vectorized_tweets[0:200000]
vectorized_cv_tweets = vectorized_tweets[200000:]

In [ ]:
np.savez_compressed("./vectorized_train_tweets.npz",*vectorized_train_tweets)
np.savez_compressed("./vectorized_cv_tweets.npz",*vectorized_cv_tweets)

In [ ]:
!mv /content/vectorized_train_tweets.npz /content/drive/MyDrive
!mv /content/vectorized_cv_tweets.npz /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/vectorized_train_tweets.npz /content
!cp /content/drive/MyDrive/vectorized_cv_tweets.npz /content

In [ ]:
filtered_reduced_data["target"] = filtered_reduced_data["target"].apply(lambda x: str(x))

In [ ]:
tweet_labels = list(filtered_reduced_data["target"].replace(to_replace=filtered_reduced_data["target"].unique(),
                                        value=list(range(len(filtered_reduced_data["target"].unique())))))

In [ ]:
train_labels = tweet_labels[0:200000]
cv_labels = tweet_labels[200000:]

In [ ]:
train_mb_size = 1000
num_epochs = 10
train_size = 200000

In [ ]:
loaded_vectorized_train_tweets = np.load("./vectorized_train_tweets.npz")

In [ ]:
def train_datagen():

  for _ in range(num_epochs):

    for i in range(train_size//train_mb_size):

      tweets_mb_list = [loaded_vectorized_train_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*train_mb_size,(i+1)*train_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in train_labels[i*train_mb_size:(i+1)*train_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in train_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

In [ ]:
cv_mb_size = 1577
cv_size = 14193

In [ ]:
loaded_vectorized_cv_tweets = np.load("./vectorized_cv_tweets.npz")

In [ ]:
def cv_datagen():

  for _ in range(num_epochs):

    for i in range(cv_size//cv_mb_size):

      tweets_mb_list = [loaded_vectorized_cv_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*cv_mb_size,(i+1)*cv_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in cv_labels[i*cv_mb_size:(i+1)*cv_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in cv_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip /content/glove.6B.zip

In [ ]:
max_vocabulary_size = len(loaded_vectorization_layer.get_vocabulary())
embedding_output_dim = 50

In [ ]:
def create_bin_class_rnn():

  rnn_model = Sequential()

  rnn_model.add(Input(shape=(None,),dtype="int64"))
  rnn_model.add(Embedding(input_dim=max_vocabulary_size,output_dim=embedding_output_dim,input_length=max_sequence_len))
  rnn_model.add(LSTM(units=50))
  rnn_model.add(Dense(units=1,activation="sigmoid"))

  return rnn_model

In [ ]:
rnn_model = create_bin_class_rnn()

In [ ]:
rnn_model.summary()

In [ ]:
rnn_model.compile(loss="binary_crossentropy",metrics=["accuracy",
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall()])

In [ ]:
training_data_gen = train_datagen()
cv_data_gen = cv_datagen()

In [ ]:
rnn_model.fit(training_data_gen,
              epochs=num_epochs,
              validation_data=cv_data_gen,
              steps_per_epoch=200,validation_steps=9)

In [ ]:
training_data_gen = train_datagen()
cv_data_gen = cv_datagen()